In [121]:
import undetected_chromedriver as uc
from time import sleep as time_sleep
from bs4 import BeautifulSoup as BS
import pandas as pd
from os.path import join as os_path_join
from os import mkdir as os_mkdir

In [122]:
url = '''

https://www.webnovel.com/stories/novel?gender=1

'''.strip()
host = '/'.join(url.split('/', maxsplit=3)[:3])
host

'https://www.webnovel.com'

In [123]:
driver = uc.Chrome()
driver.get(url)

In [124]:
for _ in range(50):
    (driver.execute_script("window.scrollBy(0,1750)"))
    time_sleep(1)

In [125]:
page = driver.page_source
soup = BS(page, 'html.parser')

In [126]:
table_0 = soup.find(attrs={'class':'f1'}).find(attrs={'class':'j_category_wrapper'}).find(attrs={'class':'clearfix'})
table = table_0.find_all(attrs={'class':r'fl w50% bsbb pl20 pr20 pt8 pb8'}) 

In [127]:
df = pd.DataFrame()

elem: BS
for index, elem in enumerate(table[:]):
    
    name_and_href = elem.find('div').find('h3', attrs={'class':r'fw700 lh20 fs16 ls0.15 ell mb4'}).find('a')
    name = name_and_href.get('title')
    href = host + name_and_href.get('href')
    try:
        description = elem.find('div').find('p', attrs={'class':r'fw400 lh20 fs14 ls0.2 c_l ells _2 mt4'}).text
    except AttributeError as ex:
        continue
        
    information = elem.find('div').find('p', attrs={'class':r'df aic'}).find_all('strong', attrs={'class':r'c_m fs0 ff_number vam mr16'})
    cnt = False
    inf: BS
    for inf in information:
        tmp = inf.find('span').text
        if 'Chapters' in tmp:
            capters = int(tmp.replace(' Chapters', ''))
            if not cnt:
                rating = float(0)    
            break
        else:
            rating = float(tmp)
            cnt = True
    
    df_temp = pd.DataFrame({
        'name':[name],
        'chapters':[capters],
        'rating':[rating],
        'href':[href],
        'description':[description]
    })
    
    df = pd.concat([df,df_temp], ignore_index=True, sort=False)
df.head()

,name,chapters,rating,href,description
0,Supreme Magus,2379,4.82,https://www.webnovel.com/book/supreme-magus_12...,Derek McCoy was a man who spent his entire lif...
1,My Vampire System,2137,4.60,https://www.webnovel.com/book/my-vampire-syste...,The human Race is at war with the Vicious Dalk...
2,Blood Warlock: Succubus Partner in the Apocalypse,1236,4.71,https://www.webnovel.com/book/blood-warlock-su...,The legendary mana finally reached planet Eart...
3,Shadow Slave,756,4.80,https://www.webnovel.com/book/shadow-slave_221...,"Growing up in poverty, Sunny never expected an..."
4,The Mech Touch,4613,4.65,https://www.webnovel.com/book/the-mech-touch_1...,"After obtaining the Mech Designer System, Ves ..."


In [134]:
mask1 = df['rating'] > 4.5
mask2 = df['chapters'] > 400
df = df[mask1 & mask2].sort_values(by='rating', ascending=False, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         205 non-null    object 
 1   chapters     205 non-null    int64  
 2   rating       205 non-null    float64
 3   href         205 non-null    object 
 4   description  205 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 8.1+ KB


---
сохранение

In [135]:
try:
    os_mkdir(os_path_join('..','data'))
except FileExistsError as ex:
    pass
df.to_csv(
    os_path_join('..','data', 'webnovel.zip'), 
    index=False,
    compression={'method':'zip', 'archive_name':'webnovel.csv'}
    )

In [139]:
df.iloc[3]['href']

'https://www.webnovel.com/book/my-three-wives-are-beautiful-vampires._20553633305398405'

---

In [146]:
name = 'webnovel_rus_name'
rus_df = pd.read_csv(os_path_join('..','data', f'{name}.zip'))
rus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             205 non-null    object 
 1   name_rus         205 non-null    object 
 2   chapters         205 non-null    int64  
 3   rating           205 non-null    float64
 4   href             205 non-null    object 
 5   description      205 non-null    object 
 6   description_rus  205 non-null    object 
dtypes: float64(1), int64(1), object(5)
memory usage: 11.3+ KB


In [243]:
ind = 70
print(rus_df.iloc[ind]['name'])
print(rus_df.iloc[ind]['name_rus'], rus_df.iloc[ind]['rating'], rus_df.iloc[ind]['chapters'])
print(rus_df.iloc[ind]['description_rus'])
print(rus_df.iloc[ind]['href'])

Headed by a Snake
Возглавляемый змеей 4.7 982
Я проснулся в этом мире с Системой и мечом. Без колебаний я решил выполнить задание принцессы с помощью своего оружия: Высокомерия. Хитрость. Гордость. Я возглавлю гильдию Sol Invictus. Обитатели 7 преисподних, уважайте меня. Слуги 11 небес, познайте страх. Гильдия Непокоренных, познайте страдания тренировок, чтобы вы могли вырвать победу из пасти тех, кто осмеливается сомневаться в вас. 

Это история о змее по имени Тайкондриус, просыпающейся в фантастическом мире мечей и волшебства. Есть действие, насилие, 
непримиримое убийство и высокомерная демонстрация власти. Более чем у нескольких членов гильдии Invictus делается акцент на боевом росте, а также на психологическом росте. Существует дух товарищества, а также неизбежный комедийный диалог между мужественными мужчинами. Есть даже элементы романтики и Тайкона, убегающего от ужасающих, могущественных женщин. (Знаете ли вы, что большинство самок змей поедают своих партнеров после спаривания

In [242]:
rus_df['name_rus'].head(60)

0                           Система чрезмерного разрыва
1                                    Путешествие демона
2                   Мои Три Жены - Прекрасные Вампирши.
3          Я Могу Усовершенствовать Свой Талант Во Всем
4     Возрожденный как Призрак: Время Создавать Мою ...
5     Что вы имеете в виду, мои милые ученики - Янде...
6                                    Эпопея с гусеницей
7     Совершенно Новая Жизнь Онлайн: Восстание Богин...
8               Код Зулу Альфа: Ботаник в Апокалипсисе!
9     Культиватор с современным искусственным интелл...
10                        Сильнейший Некромант Врат Рая
11      Моя девушка - искательница приключений S-класса
12                                  Re: Эволюция онлайн
13                         Гарем эсперов в Апокалипсисе
14                                Скучающий бессмертный
15                                      Повелитель тайн
16                                 Влияние богов онлайн
17                                   Выращивание